In [1]:
import pandas as pd
import numpy as np
import seaborn as sb

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, SparsePCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score

In [3]:
import mlflow
import mlflow.sklearn
mlflow.set_experiment("P609_Economical_Survey")

2026/01/19 22:57:39 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/19 22:57:39 INFO mlflow.store.db.utils: Updating database tables
2026/01/19 22:57:39 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/19 22:57:39 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/19 22:57:41 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/19 22:57:41 INFO alembic.runtime.migration: Will assume non-transactional DDL.


<Experiment: artifact_location='file:C:/Users/YOGITA/EXCELR_DS_PROJECT/mlruns/2', creation_time=1766053288872, experiment_id='2', last_update_time=1766053288872, lifecycle_stage='active', name='P609_Economical_Survey', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [4]:
import warnings
warnings.filterwarnings('ignore')

#### Load cleaned, complete dataset

In [5]:
df = pd.read_csv("complete_World_development_mesurement.csv")
df.tail(3)

,Unnamed: 0.1,Unnamed: 0,Birth Rate,Business Tax Rate,CO2 Emissions,Country,Days to Start Business,Ease of Business,Energy Usage,GDP,...,Life Expectancy Male,Mobile Phone Usage,Number of Records,Population 0-14,Population 15-64,Population 65+,Population Total,Population Urban,Tourism Inbound,Tourism Outbound
2571,2700,2700,0.013,464.0,5681664.0,189,5.000,4.0,2132446.0,1.624460e+13,...,76.0,1.0,1,0.196,0.667,0.1355,313873685.0,0.811,2.000920e+11,1.265730e+11
2572,2701,2701,0.015,419.0,5776.0,190,40.625,85.0,3132.0,5.000435e+10,...,74.0,1.5,1,0.220,0.639,0.1400,3395253.0,0.948,2.222000e+09,7.075000e+08
2573,2702,2702,0.020,619.0,181630.0,193,141.000,180.0,63319.5,3.812862e+11,...,72.0,1.0,1,0.288,0.652,0.0600,29954782.0,0.889,9.040000e+08,3.202000e+09


In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Country'] = le.fit_transform(df['Country'])

In [7]:
old_df = df

df = df[['Business Tax Rate', 
       'Days to Start Business', 'Ease of Business', 'Energy Usage', 'GDP',
        'Hours to do Tax',
        'Internet Usage', 'Lending Interest',
        'Mobile Phone Usage',
         'Population 15-64',
       'Population Total',
       'Tourism Inbound', 'Tourism Outbound'] ]
df

,Business Tax Rate,Days to Start Business,Ease of Business,Energy Usage,GDP,Hours to do Tax,Internet Usage,Lending Interest,Mobile Phone Usage,Population 15-64,Population Total,Tourism Inbound,Tourism Outbound
0,729.000,25.000,151.0,26998.0,5.479006e+10,451.0,0.0,0.0825,0.0,0.619,31719449.0,1.020000e+08,1.930000e+08
1,521.000,93.500,178.0,7499.0,9.129595e+09,272.0,0.0,1.0320,0.0,0.499,13924930.0,3.400000e+07,1.460000e+08
2,732.000,31.000,175.0,1983.0,2.359122e+09,270.0,0.0,0.0000,0.0,0.517,6949366.0,7.700000e+07,5.000000e+07
3,171.000,92.000,65.0,1836.0,5.788312e+09,140.0,0.0,0.1550,0.1,0.587,1755375.0,2.270000e+08,2.090000e+08
4,448.500,17.000,154.0,0.0,2.610959e+09,270.0,0.0,0.0000,0.0,0.505,11607944.0,2.300000e+07,3.000000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,279.000,694.000,165.0,705.0,5.012121e+09,199.0,0.3,0.1170,1.1,0.656,534541.0,7.900000e+07,5.800000e+07
2570,291.375,41.000,63.0,18221.5,2.343634e+10,210.0,0.6,0.0770,1.4,0.705,1337439.0,5.525000e+08,1.505000e+08
2571,464.000,5.000,4.0,2132446.0,1.624460e+13,175.0,0.8,0.0330,1.0,0.667,313873685.0,2.000920e+11,1.265730e+11
2572,419.000,40.625,85.0,3132.0,5.000435e+10,310.0,0.5,0.1120,1.5,0.639,3395253.0,2.222000e+09,7.075000e+08


In [8]:
df.columns

Index(['Business Tax Rate', 'Days to Start Business', 'Ease of Business',
       'Energy Usage', 'GDP', 'Hours to do Tax', 'Internet Usage',
       'Lending Interest', 'Mobile Phone Usage', 'Population 15-64',
       'Population Total', 'Tourism Inbound', 'Tourism Outbound'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2574 entries, 0 to 2573
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Business Tax Rate       2574 non-null   float64
 1   Days to Start Business  2574 non-null   float64
 2   Ease of Business        2574 non-null   float64
 3   Energy Usage            2574 non-null   float64
 4   GDP                     2574 non-null   float64
 5   Hours to do Tax         2574 non-null   float64
 6   Internet Usage          2574 non-null   float64
 7   Lending Interest        2574 non-null   float64
 8   Mobile Phone Usage      2574 non-null   float64
 9   Population 15-64        2574 non-null   float64
 10  Population Total        2574 non-null   float64
 11  Tourism Inbound         2574 non-null   float64
 12  Tourism Outbound        2574 non-null   float64
dtypes: float64(13)
memory usage: 261.6 KB


#### Check correlation

In [10]:
corr_matrix = df.corr(numeric_only=True).abs()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
max_corr = upper_triangle.max().max()
high_corr = max_corr >= 0.8

if high_corr:
    print("Correlation is high")
else:
    print("no correlation")

Correlation is high


In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

steps = [('scaler', StandardScaler()), ('pca', PCA(n_components=0.95))]
pipeline = Pipeline(steps)
scaled_pca_data = pipeline.fit_transform(df)
print("Transformed shape:\n", scaled_pca_data.shape)

Transformed shape:
 (2574, 9)


In [12]:
steps = [('scaler', StandardScaler()), ('spca', SparsePCA(n_components=2, alpha=0.01, random_state=42))]
pipeline = Pipeline(steps)
scaled_pca_data = pipeline.fit_transform(df)
print("Transformed shape:\n", scaled_pca_data.shape)

Transformed shape:
 (2574, 2)


In [13]:
score_df = pd.DataFrame()
results = []

### KMeans Clustering Algorithms

#### try range 2 to 50 to find out best n_cluster and silhoutte score

In [14]:
# find out number of clusters using elbow method
# and plot the Within-Cluster Sum of Squares (WCSS) against different values of k.

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

wcss = []
silhouette_scores = []
for k in range(2, 50):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(scaled_pca_data)   # use PCA-transformed data    
    wcss.append(kmeans.inertia_)
    
    score = silhouette_score(scaled_pca_data, kmeans.labels_)    
    silhouette_scores.append(score)


the rate of decrease sharply slows at 15 but silhoutte score is better at 3
so, let's take n_clusters= 3

#### perform KMeans with selected c_clusters=3 and visualize 

In [15]:
from sklearn.cluster import KMeans
#with mlflow.start_run(run_name="kmeans"):
for n in range(2,5):    
    kmeans = KMeans(n_clusters=n, random_state=42)
    kmeans.fit_predict(scaled_pca_data)
    df['KMeans'] = kmeans.labels_
    score = silhouette_score(scaled_pca_data, kmeans.labels_)
    dbi = davies_bouldin_score(scaled_pca_data,  kmeans.labels_)
    chi = calinski_harabasz_score(scaled_pca_data,  kmeans.labels_)
    
    plt.figure(figsize=(8,6))
    sb.scatterplot(x=scaled_pca_data[:,0], y=scaled_pca_data[:,1], hue=kmeans.labels_, palette="Set2")
    plt.title(f"KMeans Clustering {n} (Silhouette: {score:.3f})")
    plt.savefig(f"kmeans_{n}.png")  
    plt.close()      
    with mlflow.start_run(run_name=f"kmeans_{n}"):
        mlflow.log_param("n_clusters", n)
        mlflow.log_metric("SIL", float(score))
        mlflow.log_metric("DBI", float(dbi))
        mlflow.log_metric("CHI", float(chi))              
        mlflow.log_artifact(f"kmeans_{n}.png")
        
        mlflow.sklearn.log_model(kmeans, name="KMeans")

2026/01/19 22:59:04 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



NOTE: clusters are in irregular shapes which means KMeans is not best suitable. <br>
Let's try DBSCAN, it can detect arbitrary shapes and explicitly mark outliers.

### GaussianMixture Algorithm

In [16]:
from sklearn.mixture import GaussianMixture
#with mlflow.start_run(run_name="gmm"):
# Step 3: Fit GMM
for cov in ('tied', 'spherical'):
    for n in range(2,5):
        gmm = GaussianMixture(n_components=n, covariance_type=cov, random_state=42)
        labels = gmm.fit_predict(scaled_pca_data)
        df['GMM'] = labels
        # Step 4: Compute metrics
        sil_score = silhouette_score(scaled_pca_data, labels)
        dbi = davies_bouldin_score(scaled_pca_data,  labels)
        chi = calinski_harabasz_score(scaled_pca_data,  labels)
                
        plt.figure(figsize=(8,6))
        sb.scatterplot(x=scaled_pca_data[:,0], y=scaled_pca_data[:,1], hue=labels, palette="Set2")
        plt.title(f"GMM Clustering {cov}_{n} (Silhouette: {sil_score:.3f})")
        plt.savefig(f"GMM_{cov}_{n}.png")  
        plt.close()  
        with mlflow.start_run(run_name=f"GMM_{cov}_{n}"):
            mlflow.log_param("n_components", n)
            mlflow.log_param("covariance_type", cov)
            mlflow.log_metric("SIL", float(sil_score))
            mlflow.log_metric("DBI", float(dbi))
            mlflow.log_metric("CHI", float(chi))
            mlflow.log_artifact(f"GMM_{cov}_{n}.png") 
            mlflow.sklearn.log_model(gmm, name="GMM")

### AgglomerativeClustering Algorithm

In [17]:
from sklearn.cluster import AgglomerativeClustering
#with mlflow.start_run(run_name="agglo"):
    # Step 4: Fit Agglomerative Clustering
for n in range(2,5):    
    agg = AgglomerativeClustering(n_clusters=n, linkage='complete')
    labels = agg.fit_predict(scaled_pca_data)
    df['Agglomerative'] = agg.labels_
    # Step 5: Compute metrics
    sil_score = silhouette_score(scaled_pca_data, labels)
    dbi = davies_bouldin_score(scaled_pca_data, labels)
    chi = calinski_harabasz_score(scaled_pca_data, labels)
        
    plt.figure(figsize=(8,6))
    sb.scatterplot(x=scaled_pca_data[:,0], y=scaled_pca_data[:,1], hue=labels, palette="Set2")
    plt.title(f"Agglomerative Clustering {n} (Silhouette: {sil_score:.3f})")
    plt.savefig(f"agglo_{n}.png")    
    plt.close()  
    with mlflow.start_run(run_name=f"agglo_{n}"):
        mlflow.log_param("n_clusters", n)
        mlflow.log_param("linkage", 'complete')    
        mlflow.log_metric("SIL", float(sil_score))
        mlflow.log_metric("DBI", float(dbi))
        mlflow.log_metric("CHI", float(chi))
        
        mlflow.log_artifact(f"agglo_{n}.png")        
        
        mlflow.sklearn.log_model(agg, name="AGGLO")

2026/01/19 23:04:02 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:04:31 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:04:58 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


### SpectralClustering Algorithm

In [18]:
from sklearn.cluster import SpectralClustering
# Step 4: Fit Spectral Clustering
#with mlflow.start_run(run_name="spectral"):
for aff in ('rbf', 'nearest_neighbors'):
    for n in range(2,5):
        spectral = SpectralClustering(n_clusters=n, affinity=aff, random_state=42)
        labels = spectral.fit_predict(scaled_pca_data)
        df['Spectral'] = spectral.labels_
        # Step 5: Compute metrics
        sil_score = silhouette_score(scaled_pca_data, labels)
        dbi = davies_bouldin_score(scaled_pca_data, labels)
        chi = calinski_harabasz_score(scaled_pca_data, labels)
        plt.figure(figsize=(8,6))
        sb.scatterplot(x=scaled_pca_data[:,0], y=scaled_pca_data[:,1], hue=labels, palette="Set2")
        plt.title(f"Spectral Clustering {n}_{aff} (Silhouette: {sil_score:.3f})")
        plt.savefig(f"spectral_{n}_{aff}.png")   
        plt.close()  
        with mlflow.start_run(run_name=f"spectral_{n}_{aff}"):
            mlflow.log_param("n_clusters", n)
            mlflow.log_param("affinity", aff)
            mlflow.log_metric("SIL", float(sil_score))
            mlflow.log_metric("DBI", float(dbi))
            mlflow.log_metric("CHI", float(chi))
                      
            mlflow.log_artifact(f"spectral_{n}_{aff}.png")
            
            mlflow.sklearn.log_model(agg, name="Spectral")

2026/01/19 23:05:33 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:06:08 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:06:41 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:07:16 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:07:47 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:07:59 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


### MeanShift Algorithm

In [19]:
from sklearn.cluster import MeanShift
# Step 4: Fit Mean-Shift
with mlflow.start_run(run_name="mshift"):
    ms = MeanShift()
    labels = ms.fit_predict(scaled_pca_data)
    df['MeanShift'] = ms.labels_
    # Step 5: Compute metrics
    # Note: If Mean-Shift finds only 1 cluster, metrics like silhouette won't be valid
    n_clusters = len(np.unique(labels))
    if n_clusters > 1:
        sil_score = silhouette_score(scaled_pca_data, labels)
        dbi = davies_bouldin_score(scaled_pca_data, labels)
        chi = calinski_harabasz_score(scaled_pca_data, labels)
        print(f"Mean-Shift Silhouette Score: {sil_score:.3f}")
        
        mlflow.log_metric("SIL", float(sil_score))
        mlflow.log_metric("DBI", float(dbi))
        mlflow.log_metric("CHI", float(chi))
        plt.figure(figsize=(8,6))
        sb.scatterplot(x=scaled_pca_data[:,0], y=scaled_pca_data[:,1], hue=labels, palette="Set2")
        plt.title(f"Mean-Shift Clustering (Clusters: {n_clusters})")
        plt.savefig(f"mean_shift_{n_clusters}.png")  
        plt.close()  
        mlflow.log_artifact(f"mean_shift_{n_clusters}.png")
        
        mlflow.sklearn.log_model(ms, name="MeanShift")
    else:
        print("Mean-Shift found only one cluster; metrics are not meaningful.")

Mean-Shift Silhouette Score: 0.413


### DBSCAN Algorithms

In [20]:
from sklearn.cluster import DBSCAN
#with mlflow.start_run(run_name="dbscan"):
for msample in range(9,14):
    for ep in range(2, 5):
        dbscan = DBSCAN(eps=ep, min_samples=msample)
        df['dbscan'] = dbscan.fit_predict(scaled_pca_data)
        
        if len(df['dbscan'].value_counts()) > 1:
            score = silhouette_score(scaled_pca_data, dbscan.labels_)
            dbi = davies_bouldin_score(scaled_pca_data, dbscan.labels_)
            chi = calinski_harabasz_score(scaled_pca_data, dbscan.labels_)
        else:
            score = 0
            dbi = 0
            chi = 0
            
        plt.figure(figsize=(8,6))
        labels = dbscan.labels_        
        sb.scatterplot(x=scaled_pca_data[:,0], y=scaled_pca_data[:,1], hue=dbscan.labels_, palette="Set2")
        # Plot means
        unique_labels = set(labels) - {-1}  # exclude noise
        
        cluster_means = {}
        for lbl in unique_labels:
            cluster_points = scaled_pca_data[labels == lbl]
            cluster_means[lbl] = np.mean(cluster_points, axis=0)
        
        if len(cluster_means) > 1:
            plt.scatter(cluster_means[0][0], cluster_means[1][1], c='red', s=200, alpha=0.7, marker='o')                               
    
        plt.title(f"DBSCAN Clustering (Clusters: {ep}_{msample})")
        plt.savefig(f"dbscan_{ep}_{msample}.png")     
        plt.close()                   
        
        with mlflow.start_run(run_name=f"dbscan_{ep}_{msample}"):
            mlflow.log_param("eps", ep)
            mlflow.log_param("min_samples", msample)
            mlflow.log_metric("SIL", float(score))
            mlflow.log_metric("DBI", float(dbi))
            mlflow.log_metric("CHI", float(chi))
            mlflow.log_artifact(f"dbscan_{ep}_{msample}.png")  
            mlflow.sklearn.log_model(dbscan, name="DBSCAN")    

2026/01/19 23:09:42 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:10:06 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:10:20 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:10:43 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:11:08 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:11:23 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:11:44 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:12:09 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2026/01/19 23:12:28 WARNING mlflow.sklearn: Model was missing fu

In [21]:
mlflow.end_run()